In [74]:
#import statements
import json
import numpy as np
import random
import re
import spacy

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from textblob import TextBlob

In [20]:
# get training dataset 
with open("train-data-prepared.json", "r") as f:
    train_data = json.load(f)
# get training dataset 
with open("val-data-prepared.json", "r") as f:
    val_data = json.load(f)
# get testing dataset 
with open("val-data-prepared.json", "r") as f:
    test_data = json.load(f)


In [21]:
#create spacy object
nlp_english = spacy.load("en_core_web_sm")
#create Stemmer object
stemmer = PorterStemmer()

In [46]:
entire_dataset = {
    'train_ids': [thread["id"] for thread in train_data],
    'train_posts': [thread["preceding_posts"] for thread in train_data],
    'train_label': [thread["label"] for thread in train_data],
    
    'val_ids': [thread["id"] for thread in val_data],
    'val_posts': [thread["preceding_posts"] for thread in val_data],
    'val_label': [thread["label"] for thread in val_data],
    
    'test_ids': [thread["id"] for thread in test_data],
    'test_posts': [thread["preceding_posts"] for thread in test_data],
    'test_label': [thread["label"] for thread in test_data],
}

print(entire_dataset.keys())

dict_keys(['train_ids', 'train_text', 'train_label', 'val_ids', 'val_text', 'val_label', 'test_ids', 'test_text', 'test_label'])


In [52]:
#for thread in val_data:
#    authors = []
#    for comment in thread["preceding_posts"]:
#        if comment["author_name"] not in authors:
#            authors.append(comment["author_name"])
#    
#    if len(authors) >= 2:
#        print(thread["id"])

In [65]:
# remove punctuation, space, urls from text
def clean_text(text):
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    parsed_text = nlp_english(text)
    clean_text = []
    for token in parsed_text:
        stop_flag = (token.is_punct or token.is_space or  
                 token.like_url)
        if (not stop_flag):
            clean_text.append(token.text.lower())
    return clean_text

def stem_text(text):
    return [stemmer.stem(word) for word in clean_text(text)]

print(stem_text("idiot stupid ass crazy screw you !"))
print(clean_text("> a) right, because women are non-sexual creatures who would never use prostitutes themselves\n\ni think you vastly overestimate the number of women that pay for sex..."))

['idiot', 'stupid', 'ass', 'crazi', 'screw', 'you']
['a', 'right', 'because', 'women', 'are', 'non', 'sexual', 'creatures', 'who', 'would', 'never', 'use', 'prostitutes', 'themselves', 'i', 'think', 'you', 'vastly', 'overestimate', 'the', 'number', 'of', 'women', 'that', 'pay', 'for', 'sex']


In [47]:
# checks if the OP has addressed the other person in some way
def check_insults(text):
    return [
        1 if "ass" in text else 0,
        1 if "idiot" in text else 0,
        1 if "screw" in text else 0,
        1 if "stupid" in text else 0
    ]
        
#print(check_insults(clean_text("> a) right, because women are non-sexual creatures who would never use prostitutes themselves\n\ni think you vastly overestimate the number of women that pay for sex...")))

[0, 0, 0, 0]


In [91]:
# should return one feature vector for one string
# features -> Author's turn, length of argument, insults, sentiment
def gather_data(thread, train_data_flag = False):
    returnObj = {}
    for i in range(len(thread["preceding_posts"])):
        comment_data = {}
        comment = thread["preceding_posts"][i]
        # clean text
        comment_data["text"] = stem_text(comment["body"])
        # author's identity: 
        # Assuming 3 dialogues, the authors will have indices 0,1,2
        # 0,2 would be the OP
        # 1 would be the reply
        comment_data["author_turn_vec"] = [i % 2]
        # Length just in case of Godwin's Law
        comment_data["char_length_vec"] = [len("".join(clean_text(comment["body"])))]
        # check for some common insults
        comment_data["insults_vec"] = check_insults(comment_data["text"])
        # get sentiment
        sentiment = TextBlob(' '.join(comment_data["text"])).sentiment
        comment_data["sentiment"] =  [sentiment.polarity, sentiment.subjectivity]
        feature_vec = [
            *comment_data["author_turn_vec"], 
            *comment_data["char_length_vec"], 
            *comment_data["insults_vec"], 
            *comment_data["sentiment"]
        ]
        
        returnObj[" ".join(comment_data["text"])] = feature_vec
        
    return returnObj

In [92]:
# print features for some 10 tuples
for thread in train_data[:5]:
    print(gather_data(thread))
    print()

{'a right becaus women are non sexual creatur who would never use prostitut themselv i think you vastli overestim the number of women that pay for sex': [0, 132, 0, 0, 0, 0, 0.39285714285714285, 0.6845238095238095], 'becaus it s illeg in our realiti vs the propos realiti that me op and everyon els on thi thread are comment about keep up qwert': [1, 113, 0, 0, 0, 0, 0.0, 0.0], 'i live in a nation were it is complet legal to pay for sex surround by other nation where it is also mostli legal guess what no woman pay for sex and you ll notic that it be illeg in the us doesn t stop men from go to prostitut keep up bunchanumb edit for context it s a first world nation with consider more liber attitud regard sex then the us': [0, 305, 0, 0, 0, 0, 0.19356060606060607, 0.3513888888888889]}

{'becaus make prostitut legal make it veri much more difficult to polic thi as ha been found repeatedli in countri around the world the women who are brought into the countri can t be question becaus there is

In [ ]:
entire_dataset["train_prep"] = [gather_data(thread) for thread in train_data]

entire_dataset["val_prep"] = [gather_data(thread) for thread in val_data]

entire_dataset["test_prep"] = [gather_data(thread) for thread in test_data]

In [95]:
for text in entire_dataset["train_prep"][:5]:
    print(text)

#train_bow_input = [text.keys() for text in entire_dataset["train_prep"]]
#val_bow_input = []
#test_bow_input = []

print(len(train_bow_input))

[['a right becaus women are non sexual creatur who would never use prostitut themselv i think you vastli overestim the number of women that pay for sex', [0, 132, 0, 0, 0, 0, 0.39285714285714285, 0.6845238095238095]], ['becaus it s illeg in our realiti vs the propos realiti that me op and everyon els on thi thread are comment about keep up qwert', [1, 113, 0, 0, 0, 0, 0.0, 0.0]], ['i live in a nation were it is complet legal to pay for sex surround by other nation where it is also mostli legal guess what no woman pay for sex and you ll notic that it be illeg in the us doesn t stop men from go to prostitut keep up bunchanumb edit for context it s a first world nation with consider more liber attitud regard sex then the us', [0, 305, 0, 0, 0, 0, 0.19356060606060607, 0.3513888888888889]]]
[['becaus make prostitut legal make it veri much more difficult to polic thi as ha been found repeatedli in countri around the world the women who are brought into the countri can t be question becaus th

In [ ]:
vectorizer = CountVectorizer()



In [32]:
# random classification for baseline score

random_val = {t_id: random.randint(0,1) for t_id in val_ids}
random_test = {t_id: random.randint(0,1) for t_id in test_ids}


In [33]:
with open("val-random-output.json", "w") as f:
    json.dump(random_val, f)
# get testing dataset 
with open("test-random-output.json", "w") as f:
    json.dump(random_test, f)
